## Relax Take Home Challenge

First import the pandas package

In [1]:
# First import the pandas package
import pandas as pd

Next read in the users data using pandas read_csv

In [2]:
#  Create the users dataframe
users = pd.read_csv("C://Users//karlk//takehome_users.csv", encoding = 'latin-1')

Then read in the engagement data, also using pandas read_csv

In [3]:
# Create the engagement dataframe
engagement = pd.read_csv("C://Users//karlk//takehome_user_engagement.csv", encoding = 'latin-1')

Now that the data has been read, let's look at the first few records for both the users and engagement tables.

In [4]:
# Examine the head of the users table
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,5/21/2013 8:09,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,1/17/2013 10:14,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [5]:
# Examine the head of the engagement table
engagement.head()

,time_stamp,user_id,visited
0,4/22/2014 3:53,1,1
1,11/15/2013 3:45,2,1
2,11/29/2013 3:45,2,1
3,12/9/2013 3:45,2,1
4,12/25/2013 3:45,2,1


In the cell below, the time stap is converted to a pandas datetime via to_datetime

In [6]:
engagement.time_stamp = pd.to_datetime(engagement.time_stamp)

To check if a given user has engaged the services 3 or more times in a week we can use the Grouper, groupby, and sum calls from pandas. the freq='w' indicates that we wish to group by week intervals

In [7]:
engageWeekly = engagement.groupby(['user_id', 
     pd.Grouper(key='time_stamp', freq='W')]
)['visited'].sum().reset_index().sort_values('time_stamp')

After the data is aggregated, we can filter down to just those who have engaged the desired amounts.

In [8]:
VisitsInWeek3orMore = engageWeekly[engageWeekly['visited'] >= 3].sort_values('user_id')

Below we will create a unquie user data frame for just those who have interacted 3 or more times in a week.

In [10]:
# Next, create an array of unique user_ids
import numpy as np
UniqueUser = pd.DataFrame({'user_id': np.unique(VisitsInWeek3orMore.user_id)})

In the cell below we add an adopted column and set the 3 per week users values to 1.

In [16]:
# Create a dummy variable for Unique Users
UniqueUser['adopted'] = 1

In the cell below we get a count of unique users who have engaged at least once.

In [18]:
# Count of users with any engagement
len(np.unique(engagement.user_id))

8823

Next we left outer join the UniqueUsers with 3 or more engagments per week with the original users table and fill the NaN values with 0. This is really just setting the non-adopted records to 0 instead of NaN

In [13]:
FullUnique = users.merge(UniqueUser, 
                 left_on = 'object_id', 
                 right_on = 'user_id', 
                 how = 'left').fillna(0)

Next we will create a feature of the total number of visits for each user, regardless of the time frequency.

In [19]:
# Get count of users with any engagment
engageTotal = engagement.groupby(['user_id'])['visited'].sum().reset_index()

In [20]:
engageTotal.head()

,user_id,visited
0,1,1
1,2,14
2,3,1
3,4,1
4,5,1


In [22]:
# Set the column names of the new aggregated dataframe
engageTotal.columns = ['object_id', 'TotalEngage']

In [21]:
FullUnique.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id,adopted
0,1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0.0,0.0
1,2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2.0,1.0
2,3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0.0,0.0
3,4,5/21/2013 8:09,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0.0,0.0
4,5,1/17/2013 10:14,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0.0,0.0


Below we left outer join the FullUnique user table with the total engagment table

In [23]:
FullUnique = FullUnique.merge(engageTotal, 
                 on = 'object_id',
                 how = 'left').fillna(0)

In [24]:
FullUnique.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id,adopted,TotalEngage
0,1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0.0,0.0,1.0
1,2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2.0,1.0,14.0
2,3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,0.0,0.0,1.0
3,4,5/21/2013 8:09,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,0.0,0.0,1.0
4,5,1/17/2013 10:14,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,0.0,0.0,1.0


In [25]:
# Make sure that the length of the FullUnique table is still 12000
len(FullUnique)

12000

Now we can start preparing the data to actually start modeling. First we will label encode the creation source attribute so that we can train a decision tree on the data.

In [26]:
from sklearn.preprocessing import LabelEncoder
# Instantiate a LabelEncoder object
enc = LabelEncoder()
# Fit the label encoding to the creation_source attribute
enc.fit(FullUnique['creation_source'])
# Create a new attribute for the transformed label encoded variable
FullUnique['cs_LabelEnc'] = enc.transform(FullUnique['creation_source'])

Below we create the test and training sets to model the data.

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
x = FullUnique[['cs_LabelEnc',
                'opted_in_to_mailing_list', 
                'enabled_for_marketing_drip',
                'TotalEngage']]
y = (FullUnique['adopted'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=100)

First Lets train a Logistic regression.

In [53]:
# Instantiate the Logistic Regression object
LR = LogisticRegression()
# Fit the model to the x and y training variables
LR.fit(x_train, y_train)
# Predict the test variables
Pred= LR.predict(x_test)

Finally we can look at the actual results from the Logistic regression.

In [54]:
print('Confusion matrix for Decision Tree Classifier with criterion gini index')
print(confusion_matrix(y_test, Pred))
print('Test accuracy: '+str(accuracy_score(y_test, Pred)))

Confusion matrix for Decision Tree Classifier with criterion gini index
[[2110   17]
 [  17  256]]
Test accuracy: 0.985833333333


As a rough guide line for the feature importance, we can look at the size or influence of the coefficients.

In [82]:
for i in range(0, len(LR.coef_[0])):
    print(str(x_train.columns[i]) + ',\n Coefficient:  ' + str(LR.coef_[0][i]))

#clf.feature_importances_

cs_LabelEnc,
 Coefficient:  -0.0014280184646
opted_in_to_mailing_list,
 Coefficient:  0.0934818097461
enabled_for_marketing_drip,
 Coefficient:  -0.266794652412
TotalEngage,
 Coefficient:  0.513005346485


From the cell above we can see that the total engagement is the most important coefficient, followed by enable for marketing drip, then opt in for mailing, and finally the creation source.

Below we will also create a decision tree classifier to predict adpotion.

In [46]:
from sklearn import tree
# Create a decision tree object
clf = tree.DecisionTreeClassifier()
# Fit the model
clf = clf.fit(x_train, y_train)
# Predict on the testing class
Pred= clf.predict(x_test)

Below are the results from the decision tree.

In [47]:
print('Confusion matrix for Decision Tree')
print(confusion_matrix(y_test, Pred))
print('Test accuracy: '+str(accuracy_score(y_test, Pred)))

Confusion matrix for Decision Tree Classifier with criterion gini index
[[2109   18]
 [  18  255]]
Test accuracy: 0.985


Finally we can check the feature importance from the decision tree.

In [64]:
for i in range(0, len(clf.feature_importances_)):
    print(str(x_train.columns[i]) + ',\n Importance:   ' + str(clf.feature_importances_[i]))

cs_LabelEnc,
 Importance:   0.0115695882329
opted_in_to_mailing_list,
 Importance:   0.00875548062926
enabled_for_marketing_drip,
 Importance:   0.00351888819892
TotalEngage,
 Importance:   0.976156042939


From the feature importance scores we can see that the attriubte Total engagement is by far the most important, followed by the source, email opt in, and finally enabled for market drip. Judging more losely by the LR coefficients the Total Engagment is still by far the most influential coefficent but the other coefficients are in different order they are as follows after total engagement enabled for market drip, email opt in and finally source.